In [1]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import math
import scipy.io as sio
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as pl
from sklearn import metrics
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary

In [2]:
file = h5py.File('D:/yrp/GW data/waveforms.h5', 'r')

In [3]:
def read_datasets(file, data_array):
    for key in file.keys():
        if isinstance(file[key], h5py.Dataset):
            data_array.append(file[key][()])
        elif isinstance(file[key], h5py.Group):
            read_datasets(file[key], data_array)

data_array = []

read_datasets(file, data_array)

file.close()

print("数据集长度:", len(data_array))
for i, data in enumerate(data_array):
    print(f"样本 {i+1} 形状:", data.shape)

数据集长度: 4500
样本 1 形状: (386581,)
样本 2 形状: (386581,)
样本 3 形状: (386581,)
样本 4 形状: (386581,)
样本 5 形状: (386581,)
样本 6 形状: (386581,)
样本 7 形状: (386581,)
样本 8 形状: (386581,)
样本 9 形状: (386581,)
样本 10 形状: (386581,)
样本 11 形状: (386581,)
样本 12 形状: (386581,)
样本 13 形状: (386581,)
样本 14 形状: (386581,)
样本 15 形状: (386581,)
样本 16 形状: (386581,)
样本 17 形状: (386581,)
样本 18 形状: (386581,)
样本 19 形状: (386581,)
样本 20 形状: (386581,)
样本 21 形状: (386581,)
样本 22 形状: (386581,)
样本 23 形状: (386581,)
样本 24 形状: (386581,)
样本 25 形状: (386581,)
样本 26 形状: (386581,)
样本 27 形状: (386581,)
样本 28 形状: (386581,)
样本 29 形状: (386581,)
样本 30 形状: (386581,)
样本 31 形状: (386581,)
样本 32 形状: (386581,)
样本 33 形状: (386581,)
样本 34 形状: (386581,)
样本 35 形状: (386581,)
样本 36 形状: (386581,)
样本 37 形状: (386581,)
样本 38 形状: (386581,)
样本 39 形状: (386581,)
样本 40 形状: (386581,)
样本 41 形状: (386581,)
样本 42 形状: (386581,)
样本 43 形状: (386581,)
样本 44 形状: (386581,)
样本 45 形状: (386581,)
样本 46 形状: (386581,)
样本 47 形状: (386581,)
样本 48 形状: (386581,)
样本 49 形状: (386581,)
样本 50 形状: (386581

In [4]:
data = np.vstack(data_array)
data

array([[-0.11062794, -0.00620342,  0.38711439, ...,  0.33523403,
        -0.0892866 , -0.21303449],
       [-0.08733686, -0.24623914, -0.14237727, ...,  0.20678462,
         0.42952339,  0.66361755],
       [-0.34400142,  0.29898094,  0.02202606, ..., -0.25228444,
        -0.02257084,  0.04270295],
       ...,
       [ 0.00813348,  0.328287  , -0.56746026, ...,  0.43614946,
        -0.30774154, -0.16574495],
       [ 0.39576865,  0.13496684,  0.32666517, ...,  0.48496836,
         0.11306705, -0.65239143],
       [-0.00197092, -0.13937108, -0.14988322, ...,  0.08421691,
         0.00827349, -0.03750114]])

In [5]:
tc_values = []

with open(r'D:/yrp/GW data/sim.log', 'r') as f:
    for line in f:
        match = re.search(r"Tc:\s*([\d.]+)", line)
        if match:
            tc_values.append(float(match.group(1)))

tc = np.array(tc_values).reshape(-1, 1)

tc

array([[ 8301187.77857865],
       [ 8301187.77857865],
       [ 8301187.77857865],
       ...,
       [11660272.61772326],
       [11660272.61772326],
       [11660272.61772326]])

In [6]:
x_train, x_valtest, y_train, y_valtest = train_test_split(data, tc, test_size=0.2, random_state=42)

x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size=0.5, random_state=42)

print("训练集:", x_train.shape, y_train.shape)
print("验证集:", x_val.shape, y_val.shape)
print("测试集:", x_test.shape, y_test.shape)

训练集: (3600, 386581) (3600, 1)
验证集: (450, 386581) (450, 1)
测试集: (450, 386581) (450, 1)


In [7]:
std = StandardScaler().fit(x_train)
x_train = std.transform(x_train)
x_val = std.transform(x_val)
x_test = std.transform(x_test)

x_train

array([[-0.18509961, -2.21848709,  0.02807074, ...,  2.64794519,
         0.9175103 , -0.84857225],
       [-1.19926415,  0.22565425, -1.75328264, ..., -0.69368186,
         1.41566747,  0.41642639],
       [-0.18007755,  0.64756451, -1.08833272, ...,  1.18414766,
        -0.64401488,  0.2936611 ],
       ...,
       [ 1.3350548 , -0.62638347, -1.38896684, ...,  0.20720433,
         0.56245029,  1.91659527],
       [-1.50383113, -1.35963624, -0.56939136, ...,  0.70836084,
         1.4209506 , -0.487472  ],
       [-0.50767206,  1.36521163,  1.15098596, ..., -0.37674884,
        -2.23300111, -1.75542209]])

In [8]:
label_std = StandardScaler().fit(y_train)
y_train = label_std.transform(y_train)
y_val = label_std.transform(y_val)
y_test = label_std.transform(y_test)

y_train = y_train.reshape(-1)
y_val = y_val.reshape(-1)
y_test = y_test.reshape(-1)

y_train

array([-1.40668927, -0.77118853, -0.23973313, ...,  2.12596539,
       -0.48187089,  0.40974714])

In [9]:
x_train = torch.unsqueeze(torch.from_numpy(x_train), dim=1).to(torch.float32)
x_val = torch.unsqueeze(torch.from_numpy(x_val), dim=1).to(torch.float32)
x_test = torch.unsqueeze(torch.from_numpy(x_test), dim=1).to(torch.float32)

y_train = torch.from_numpy(y_train).to(torch.float32)
y_val = torch.from_numpy(y_val).to(torch.float32)
y_test = torch.from_numpy(y_test).to(torch.float32)

x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

(torch.Size([3600, 1, 386581]),
 torch.Size([450, 1, 386581]),
 torch.Size([450, 1, 386581]),
 torch.Size([3600]),
 torch.Size([450]),
 torch.Size([450]))

In [11]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=7, stride=stride, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, self.expansion * out_channels, kernel_size=7, stride=stride, padding=3, bias=False),
                nn.BatchNorm1d(self.expansion * out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(512 * block.expansion, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 1)
        )

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out


def ResNetModel():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [13]:
model = ResNetModel()

In [14]:
summary(model, input_size=(1, 386581))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1           [-1, 64, 193291]             448
       BatchNorm1d-2           [-1, 64, 193291]             128
              ReLU-3           [-1, 64, 193291]               0
         MaxPool1d-4            [-1, 64, 96646]               0
            Conv1d-5            [-1, 64, 96646]          28,672
       BatchNorm1d-6            [-1, 64, 96646]             128
              ReLU-7            [-1, 64, 96646]               0
            Conv1d-8            [-1, 64, 96646]          12,288
       BatchNorm1d-9            [-1, 64, 96646]             128
             ReLU-10            [-1, 64, 96646]               0
       BasicBlock-11            [-1, 64, 96646]               0
           Conv1d-12            [-1, 64, 96646]          28,672
      BatchNorm1d-13            [-1, 64, 96646]             128
             ReLU-14            [-1, 64

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)

            val_loss += loss.item() * inputs.size(0)

    val_loss /= len(val_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "D:/yrp/GW data/best_model.pt")

Epoch 1/100, Train Loss: 1.0105, Val Loss: 1.0060
Epoch 2/100, Train Loss: 1.0000, Val Loss: 1.0414
Epoch 3/100, Train Loss: 1.0025, Val Loss: 1.0108
Epoch 4/100, Train Loss: 1.0025, Val Loss: 1.0082
Epoch 5/100, Train Loss: 0.9997, Val Loss: 7.4639
Epoch 6/100, Train Loss: 0.9994, Val Loss: 4.4268
Epoch 7/100, Train Loss: 0.9978, Val Loss: 7.4899
Epoch 8/100, Train Loss: 0.9997, Val Loss: 24.6517
Epoch 9/100, Train Loss: 0.9941, Val Loss: 1.1205
Epoch 10/100, Train Loss: 0.9929, Val Loss: 1.0009
Epoch 11/100, Train Loss: 0.9759, Val Loss: 1.0368
Epoch 12/100, Train Loss: 0.9606, Val Loss: 1.0517
Epoch 13/100, Train Loss: 0.9168, Val Loss: 1.1236
Epoch 14/100, Train Loss: 0.8555, Val Loss: 1.1966
Epoch 15/100, Train Loss: 0.7906, Val Loss: 1.3353
Epoch 16/100, Train Loss: 0.7159, Val Loss: 1.3641
Epoch 17/100, Train Loss: 0.6270, Val Loss: 1.9733
Epoch 18/100, Train Loss: 0.5510, Val Loss: 1.3444
Epoch 19/100, Train Loss: 0.4359, Val Loss: 2.6816
Epoch 20/100, Train Loss: 0.3712, Val L

In [ ]:
model.load_state_dict(torch.load("D:/yrp/GW data/best_model.pt"))

model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        test_outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)

        test_loss += loss.item() * inputs.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
test_Tc = np.array(y_test)
predict_Tc = np.array(test_outputs)
n_test = test_Tc.shape[0]

def demo_plot(x_maxsize):
    pl.figure()
    
    pl.plot(np.arange(n_test), test_Tc, label="test Tc")
    pl.plot(np.arange(n_test), predict_Tc, label="predict Tc")
    
    pl.legend()
    pl.xlabel('test samples Tc')
    pl.title('predict Tc')
    
    # change x internal size
    pl.gca().margins(x=0)
    pl.gcf().canvas.draw()
    
    # set size
    maxsize = x_maxsize
    m = 0.2
    N = 100
    s = maxsize / pl.gcf().dpi * N + 2 * m
    margin = m / pl.gcf().get_size_inches()[0]
    
    pl.gcf().subplots_adjust(left=margin, right=1. - margin)
    pl.gcf().set_size_inches(s, pl.gcf().get_size_inches()[1])

    pl.show()

demo_plot(9)